# Sample for ModelMesh Serving using the KServe Python SDK

## This notebook shows how to use the KServe SDK to create, update, and delete an InferenceService

- This sample assumes ModelMesh Serving was deployed using the [https://github.com/kserve/modelmesh-serving/blob/main/docs/quickstart.md](quickstart guide).

In [1]:
from kubernetes import client
from kserve import constants
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1TFServingSpec
from kserve import V1beta1StorageSpec
from kserve import KServeClient

### The variables being set are `namespace`, `name`, `protocol_version`
- The `namespace` definition is where the InferenceService will be deployed to
- `name` will be the name of the InferenceService
- For ModelMesh, the `v2` protocol must be used since it doesn't support the default `v1` protocol

In [2]:
namespace = 'modelmesh-serving'
name='msnist-sample'
protocol_version='v2'

### Define InferenceService specifying ModelMesh as the deploymentMode

In [3]:
isvc = V1beta1InferenceService(
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name, 
        namespace=namespace,
        annotations={
            'serving.kserve.io/deploymentMode': 'ModelMesh'
        }
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                protocol_version=protocol_version,
                storage=V1beta1StorageSpec(
                    key='localMinIO',
                    path='sklearn/mnist-svm.joblib'
                )
            )
        )
    )
)

### Create InferenceService

In [4]:
kserve = KServeClient()
kserve.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-06T19:44:36Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-06-06T19:44:36Z'}],
  'name': 'msnist-sample',
  'namespace': 'modelmesh-serving',
  'resourceVersion': '3939550',
  'uid': '64c4c371-7bc8-4929-afc3-9f4506d66a0b'},
 'spec': {'predictor': {'sklearn': {'name': '',
    'protocolVersion': 'v2',
    'storage': {'key': 'localMinIO', 'path': 'skle

### Check InferenceService status after deploying
- notice how it is in a `Pending` state

In [5]:
kserve.get(name, namespace=namespace)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-06T19:44:36Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-06-06T19:44:36Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:conditions': {},
      'f:modelStatus': {'.': {},
       'f:copies': {'.': {}, 'f:failedCopies': {}},
       'f:lastFailureInfo': {'.': {},
        

### Check to see status of InferenceService

In [6]:
kserve.wait_isvc_ready(name, namespace=namespace)

### Check InferenceService status once its ready
- State should now be `Loaded`

In [8]:
kserve.get(name, namespace=namespace)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-06T19:44:36Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-06-06T19:44:36Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:components': {'.': {},
       'f:predictor': {'.': {},
        'f:grpcUrl': {},
        'f:restUrl': {},
        'f:url': {}}},
      'f:conditions'

### Patch InferenceService and point it to a different model

In [9]:
updated_spec=V1beta1InferenceServiceSpec(
    predictor=V1beta1PredictorSpec(
        tensorflow=V1beta1TFServingSpec(
            protocol_version=protocol_version,
            storage=V1beta1StorageSpec(
                key='localMinIO',
                path='tensorflow/mnist.savedmodel'
                )
            )
        )
    )

In [10]:
updated_isvc = V1beta1InferenceService(api_version= constants.KSERVE_V1BETA1,
                          kind=constants.KSERVE_KIND,
                          metadata=client.V1ObjectMeta(name=name, namespace=namespace),
                          spec=updated_spec)

In [10]:
kserve.patch(name, updated_isvc, namespace=namespace)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-05T17:48:48Z',
  'generation': 2,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:components': {'.': {},
       'f:predictor': {'.': {},
        'f:grpcUrl': {},
        'f:restUrl': {},
        'f:url': {}}},
      'f:conditions': {},
      'f:modelStatus': {'.': {},
       'f:copies': {'.': {}, 'f:failedCopies': {}, 'f:totalCopies': {}},
       'f:states': {'.': {},
        'f:activeModelState': {},
        'f:targetModelState': {}},
       'f:transitionStatus': {}},
      'f:url': {}}},
    'manager': 'manager',
    'operation': 'Update',
    'subresource': 'status',
    'time': '2023-06-05T17:49:03Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata':

In [13]:
kserve.wait_isvc_ready(name, namespace=namespace)

kserve.get(name, namespace=namespace)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-06-06T19:44:36Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-06-06T19:44:36Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:components': {'.': {},
       'f:predictor': {'.': {},
        'f:grpcUrl': {},
        'f:restUrl': {},
        'f:url': {}}},
      'f:conditions'

### Delete InferenceService

In [12]:
kserve.delete(name, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'msnist-sample',
  'group': 'serving.kserve.io',
  'kind': 'inferenceservices',
  'uid': '62b375b1-fd66-4783-b391-d4c3f8e8e6d4'}}